In [102]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests as re
import regex as reg
import pytest as test

In [103]:
movies = soup.select(".wikitable.plainrowheaders i")


In [119]:
def get_content_value(row_data):
    if row_data.find("li"):
        return [li.get_text(" ", strip=True).replace("\xa0", " ") for li in row_data.find_all("li")]
    elif row_data.find("div",attrs={"class": "plainlist"}):
        return [text for text in row_data.stripped_strings]
    else:
        return row_data.get_text(" ", strip=True).replace("\xa0", " ") 

def clean_tags(soup):
    for tag in soup.find_all(["sup", "span"]):
        tag.decompose()

def get_info_box(url):
    r = re.get(url)
    soup = bs(r.content)
    contents = soup.prettify()
    info_box = soup.find("table", class_="infobox vevent")
    info_rows = info_box.find_all("tr")
    clean_tags(soup)

    movie_info ={}
    for index, row in enumerate(info_rows):
        if index == 0:
            movie_info['title']=row.find("th").get_text(" ", strip=True)
        elif index ==1:
            continue
        else:
            content_key =row.find("th").get_text(" ", strip=True)
            content_value = get_content_value(row.find("td"))
            movie_info[content_key] = content_value

    return movie_info



In [122]:
url_list = "https://en.wikipedia.org/wiki/List_of_Marvel_Cinematic_Universe_films"
r = re.get(url_list)
soup = bs(r.content)
movies = soup.select(".wikitable.plainrowheaders i a")

base_path = "https://en.wikipedia.org/"
movie_info_list = []

for index, movie in enumerate(movies):
    link = movie['href']
    full_path = base_path + link
    title = movie['title']
    movie_info_list.append(get_info_box(full_path))

In [144]:
import json

In [145]:
def save_data(title, data):
    with open(title, 'w', encoding ='utf-8') as f:
        json.dump(data, f, ensure_ascii = False, indent=2)


In [146]:
def load_data(title):
    with open(title, encoding ='utf-8') as f:
        return json.load(f)

In [147]:
save_data('marvel.json', movie_info_list)

In [148]:
movie_info_list = load_data('marvel.json')

In [157]:
import pickle

In [158]:
def save_data_pickle(name, data):
    with open(name, 'wb') as f:
        pickle.dump(data, f)
        

In [160]:
def load_data_pickle(name):
    with open(name, 'rb') as f:
        return pickle.load(f)

In [161]:
save_data_pickle("marvel.pickle", movie_info_list)

In [163]:
data = load_data_pickle("marvel.pickle")

In [174]:
from omdbapi.movie_search import GetMovie
import urllib
import os

In [178]:
def get_omdb_info(title):
    base_url = "https://www.omdbapi.com/?"
    parameters = {"apikey": '******', "t": title}
    params_encoded = urllib.parse.urlencode(parameters)
    full_url = base_url + params_encoded
    return re.get(full_url).json()


In [180]:
for movie in movie_info_list:
    title = movie['title']
    omdb_info = get_omdb_info(title)
    movie['imdb'] = omdb_info.get('imdbRating', None)

In [185]:
final = save_data('Finalmarvel.json', movie_info_list)

In [190]:
import pandas as pd
df = pd.read_json ('Finalmarvel.json')

In [192]:
df.to_excel("FinalMarvel.xlsx", index=False)